In [16]:
import webbrowser
import json
import requests
import pandas as pd
from stravalib.client import Client
import pprint as pprint
import time


In [ ]:
client_id, client_secret = open('client_secrets.txt').read().strip().split(',')
client = Client()
request_scope = ["read_all", "profile:read_all", "activity:read_all"]

redirect_url = "http://127.0.0.1:5000/authorization"

url = client.authorization_url(
    client_id=client_id,
    redirect_uri=redirect_url,
    scope=request_scope,
)


True

In [20]:
# Open the url that you created above in a web browser
webbrowser.open(url)
print(
    """You will see a URL that looks like this:
    http://127.0.0.1:5000/authorization?state=&code=12323423423423423423423550&scope=read,activity:read_all,profile:read_all,read_all
    Copy the values between code= and & in the URL that you see in the browser."""
)


You will see a URL that looks like this:
    http://127.0.0.1:5000/authorization?state=&code=12323423423423423423423550&scope=read,activity:read_all,profile:read_all,read_all
    Copy the values between code= and & in the URL that you see in the browser.


In [ ]:
http://127.0.0.1:5000/authorization?state=&code=9bb8696751571bf6d2a5f138d2de380f56b98a67&scope=read,activity:read_all,profile:read_all,read_all

In [22]:
# Using input allows you to copy the code into your Python console (or Jupyter Notebook)
code = input("Please enter the code that you received: ")

print(
    f"Great! Your code is {code}\n"
    "Next, I will exchange that code for a token.\n"
    "I only have to do this once."
)

# Exchange the code returned from Strava for an access token
token_response = client.exchange_code_for_token(
    client_id=client_id, client_secret=client_secret, code=code
)

token_response
# Example output of token_response
# {'access_token': 'value-here-123123123', 'refresh_token': # '123123123',
# 'expires_at': 1673665980}

# Get current athlete details
athlete = client.get_athlete()
# Print athlete name :) If this works, your connection is successful!
print(f"Hi, {athlete.firstname} Welcome to stravalib!")


Great! Your code is 9bb8696751571bf6d2a5f138d2de380f56b98a67
Next, I will exchange that code for a token.
I only have to do this once.


Hi, Tom Welcome to stravalib!


In [54]:
# Save the token response as a JSON file
json_path = 'strava_token.json'
with open(json_path, "w") as f:
    json.dump(token_response, f)

print("Token saved - hooray!")

# Access and refresh tokens
access_token = token_response["access_token"]
refresh_token = token_response["refresh_token"]

Token saved - hooray!


In [17]:
def read_token():

  try:
      with open('strava_token.json', 'r') as f:
          token = json.load(f)
  except FileNotFoundError:

      # token cannot be found, so cannot be refreshed
      # instead, follow the original authorisation procedure again
      token = initial_authorisation_token()

  return token

In [18]:
def refresh_token(token):

    # check if the token has expired
    if token['expires_at'] < time.time():

        # request a new access token using the refresh token
        token = requests.post(url='https://www.strava.com/api/v3/oauth/token',
                             data={'client_id': client_id,
                                   'client_secret': client_secret,
                                   'grant_type': 'refresh_token',
                                   'refresh_token': token['refresh_token']})
        token = token.json()    
        write_token(token)
        
    return token

In [19]:
token

{'access_token': '0ee5eadf312833521d575769e82ecea93f52531a',
 'refresh_token': 'e33ff7367514a305c2abdea8a23789456bbd755b',
 'expires_at': 1758594125}

In [ ]:
# get & update token first, uses tools described above
token = read_token()
token = refresh_token(token)


# import all user activities

activities = []
page = 1
response = []

while True:
    
    # request new page of activities
    endpoint = f"https://www.strava.com/api/v3/athlete/activities?" \
                  f"access_token={token['access_token']}&" \
                  f"page={page}&" \
                  f"per_page=50"
    
    response = requests.get(endpoint).json()

    if len(response):
        activities += [i for i in response]

    else:
        break  

# convert our activities to a DataFrame
print(activities)

In [23]:
# get & update token first, uses tools described above
token = read_token()
token = refresh_token(token)


# import all user activities

activities = []
page = 1
response = []

while True:
    
    # request new page of activities
    endpoint = f"https://www.strava.com/api/v3/athlete/activities?" \
                  f"access_token={token['access_token']}&" \
                  f"page={page}&" \
                  f"per_page=50"
    
    response = requests.get(endpoint).json()

    # check if page contains activities
    if len(response):

        # retrieve some fields for each activity
        # you can see the full list of fields by looking at the response json
        activities += [{"name": i["name"],
                "distance": i["distance"],
                "type": i["type"],
                "sport_type": i["sport_type"],
                "moving_time": i["moving_time"],
                "elapsed_time": i["elapsed_time"],
                "date": i["start_date"],
                "polyline": i["map"]["summary_polyline"],
                "map_id": i["map"]["id"],
                "start_latlng": i["start_latlng"]} for i in response]
        page += 1  

    else:
      break  

# convert our activities to a DataFrame
df = pd.DataFrame(activities)

TypeError: string indices must be integers, not 'str'

In [66]:
df

,name,distance,type,sport_type,moving_time,elapsed_time,date,polyline,map_id,start_latlng
0,Evening Run,3009.0,Run,Run,1101,1115,2025-09-18T18:57:52Z,qfryHrxc@EzA@Ba@BIJIB[@WD_@GGMQCQNGLCTEzACdC@\...,a15857721289,"[51.544128, -0.186059]"
1,Evening Run,3007.7,Run,Run,1194,1202,2025-09-15T18:42:55Z,_fryHrwc@?lBMb@y@QMKU?c@J[OYRENCPD|CIfBBPAzB@~...,a15822819329,"[51.544077, -0.185931]"
2,Evening Workout,0.0,Workout,Workout,2267,2267,2025-09-14T17:25:19Z,,a15822813266,[]
3,Evening Run,3011.4,Run,Run,1098,1225,2025-09-12T17:45:17Z,qeryHfuc@E`EIPm@EWBUHc@?c@HWRK`F@fBIrCCDG?e@Kg...,a15788775173,"[51.544089, -0.185492]"
4,Evening Run,3009.1,Run,Run,1209,1209,2025-09-09T17:51:49Z,_fryHpuc@?hDGP_@A_@Ly@Ag@BSJKR@bAGdF?bBCjACNEB...,a15755138566,"[51.54411, -0.185531]"
...,...,...,...,...,...,...,...,...,...,...
1430,Morning,10782.6,Ride,Ride,2100,2100,2017-09-29T08:00:52Z,,a1207483623,[]
1431,Evening Ride,10800.3,Ride,Ride,2104,2229,2017-09-28T17:11:49Z,qobyHbzU^gEkAPA~AxFh`@iAln@cFpJxLve@~MvUth@bf@...,a1206069250,"[51.463774, -0.116975]"
1432,Afternoon Swim,1000.0,Swim,Swim,2220,2220,2017-09-28T13:57:58Z,,a1205759657,[]
1433,To work,10239.3,Ride,Ride,1976,1976,2017-09-28T07:34:32Z,{c{xH`uh@DsBsCzC_HaXcIyHmDwIHsCcDeLuO_HyEob@_L...,a1205349791,"[51.426065, -0.213448]"
